Goal of this notebook:

Tuning the n parameter for IGAR attribute selection algorithm:

Average number of attributes among the projects: 169.16

Minimum number of attributes among the projects: 84


## Tuning strategy

### Collect accuracy for each n

- Vary the n parameter from 1 to 84 (min attributes)
- For each n:
	- For each project:
		- Execute the IGAR using n
		- Execute the prediction (calculate accuracy and normalized improvement)


### Calculate the number of wins for each n:

- For each project:
	- Check which n achieved the best accuracy

IGAR reference: https://www.cs.waikato.ac.nz/~mhall/HallHolmesTKDE.pdf

In [1]:
import numpy as np
import pandas as pd
from IPython.core.display import display

pd.set_option('display.max_columns', None)
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFECV
import warnings
import classifier_utils
warnings.filterwarnings("ignore", category=Warning)
import configs

In [2]:
non_features_columns = ["chunk_id", "line_start", "line_end", "line_separator", "kind_conflict", "url", "project"]
non_features_columns.extend(["project_user", "project_name", "path", "file_name", "sha", "leftsha", "rightsha", "basesha"])

In [3]:
selected_dataset = pd.read_csv("../../data/SELECTED_LABELLED_DATASET.csv")
projects = list(selected_dataset['project'].unique())

In [4]:
rf = RandomForestClassifier(random_state=99, n_jobs=-1, n_estimators=400, max_features=0.3, min_samples_leaf=1)

In [5]:
min_n = 0 # min_n = 0 means no selection at all, used as a baseline
max_n = 84

In [6]:
import importlib
importlib.reload(classifier_utils)
tuning_results = classifier_utils.IGAR_tuning(rf, projects, non_features_columns, min_n, max_n, True)

In [7]:
tuning_results

,project,n,accuracy,accuracy_selected,orig_attr
0,Ramblurr__Anki-Android,0,0.729897,0.729897,129
1,apache__directory-server,0,0.938673,0.938673,96
2,android__platform_frameworks_base,0,0.816667,0.816667,566
3,freenet__fred,0,0.669000,0.669000,134
4,alexo__wro4j,0,0.578225,0.578225,107
...,...,...,...,...,...
1695,apache__accumulo,84,0.863404,0.863402,137
1696,Unidata__thredds,84,0.924211,0.923158,99
1697,jgralab__jgralab,84,0.870697,0.867926,98
1698,sebastianbenz__Jnario,84,0.999021,0.999021,84


In [8]:
import importlib
importlib.reload(classifier_utils)
summary = classifier_utils.compute_IGAR_tuning_summary(tuning_results, min_n, max_n)

In [9]:
summary

,n,average_default_accuracy,average_accuracy,improvement,average_ranking,number_wins
0,0,0.801153,0.801153,0.000000,17.125,5
1,1,0.801153,0.711797,-0.111534,70.275,1
2,2,0.801153,0.718666,-0.102960,69.550,0
3,3,0.801153,0.726645,-0.093001,65.875,0
4,4,0.801153,0.730388,-0.088328,65.325,0
...,...,...,...,...,...,...
80,80,0.801153,0.797374,-0.004717,26.375,0
81,81,0.801153,0.797610,-0.004422,22.550,1
82,82,0.801153,0.797467,-0.004601,25.150,1
83,83,0.801153,0.797907,-0.004051,19.875,0


In [10]:
summary.sort_values(['number_wins', 'average_accuracy', 'average_ranking'], ascending=False)

,n,average_default_accuracy,average_accuracy,improvement,average_ranking,number_wins
0,0,0.801153,0.801153,0.000000,17.125,5
81,81,0.801153,0.797610,-0.004422,22.550,1
68,68,0.801153,0.797531,-0.004520,22.750,1
82,82,0.801153,0.797467,-0.004601,25.150,1
78,78,0.801153,0.797309,-0.004797,20.750,1
...,...,...,...,...,...,...
6,6,0.801153,0.730640,-0.088014,67.200,0
4,4,0.801153,0.730388,-0.088328,65.325,0
5,5,0.801153,0.729905,-0.088932,65.850,0
3,3,0.801153,0.726645,-0.093001,65.875,0


In [11]:
summary.sort_values(['average_ranking', 'number_wins', 'average_accuracy'], ascending=True)

,n,average_default_accuracy,average_accuracy,improvement,average_ranking,number_wins
0,0,0.801153,0.801153,0.000000,17.125,5
84,84,0.801153,0.798196,-0.003690,18.900,0
83,83,0.801153,0.797907,-0.004051,19.875,0
78,78,0.801153,0.797309,-0.004797,20.750,1
73,73,0.801153,0.796740,-0.005507,21.300,0
...,...,...,...,...,...,...
6,6,0.801153,0.730640,-0.088014,67.200,0
8,8,0.801153,0.735278,-0.082225,67.500,0
13,13,0.801153,0.740071,-0.076242,67.725,0
2,2,0.801153,0.718666,-0.102960,69.550,0
